# 모듈

In [1]:
!pip install konlpy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel
import pandas as pd
import numpy as np
import re
from konlpy.tag import Okt

# Mecab 설치

In [ ]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

In [ ]:
cd Mecab-ko-for-Google-Colab/

In [ ]:
! bash install_mecab-ko_on_colab190912.sh

In [3]:
from konlpy.tag import Mecab

# 데이터 로드

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/TF-IDF_Test/chat_data.csv')
df

,Unnamed: 0,question,answer
0,0,﻿좋은 아침.,안녕하세요.
1,1,좋은 아침.,반갑습니다.
2,2,좋은 아침.,좋은 아침이에요.
3,3,좋은 아침.,간밤에 별일 없으셨죠?
4,4,좋은 아침.,안녕하십니까?
...,...,...,...
2393,2393,"그 사람이 응급실 의사한테 뭐라고 속닥거리니까, 저보고 갑자기 응급처치 끝났다고, ...",응급실이 있어서 다행이네요. 큰 문제는 없으신 거죠?
2394,2394,파편이 튀어서 그 때 저도 응급실 가서 치료 받기도 했고…,응급실에 가셨다니 정말 놀랐어요. 아무 문제 없으신가요? 걱정 되네요.
2395,2395,지금 상태가 너무 안 좋아서 학교 안 나가고 있어요.,상태가 더 안 좋아지셨군요. 걱정이 되네요.
2396,2396,진짜 심해진 거 같긴 해요.,정말 힘드시겠어요. 지금도 증상이 심하신가요?


# 데이터 전처리

In [4]:
# 한글, 숫자, 영어 빼고 전부 제거

def text_preprocessing(text):
    sentence = text.split(' ')
    total_sentence = []
    for word in sentence:
        prepro_word = re.sub(r'[^ㄱ-ㅎㅏ-ㅣ가-힣0-9a-zA-Z]', '', word)
        total_sentence.append(prepro_word)
    result = ''.join(total_sentence)
    return result

q_prepro = df.question.map(text_preprocessing)
a_prepro = df.answer.map(text_preprocessing)
new_df = pd.concat([q_prepro, a_prepro], axis=1)
new_df

,question,answer
0,좋은아침,안녕하세요
1,좋은아침,반갑습니다
2,좋은아침,좋은아침이에요
3,좋은아침,간밤에별일없으셨죠
4,좋은아침,안녕하십니까
...,...,...
2393,그사람이응급실의사한테뭐라고속닥거리니까저보고갑자기응급처치끝났다고나가라고하더라구요,응급실이있어서다행이네요큰문제는없으신거죠
2394,파편이튀어서그때저도응급실가서치료받기도했고,응급실에가셨다니정말놀랐어요아무문제없으신가요걱정되네요
2395,지금상태가너무안좋아서학교안나가고있어요,상태가더안좋아지셨군요걱정이되네요
2396,진짜심해진거같긴해요,정말힘드시겠어요지금도증상이심하신가요


In [12]:
okt = Okt()
mecab= Mecab()

# 불용어 처리 없이 okt로 형태소 분석만 진행
def okt_no_stopwords(text):
    test = okt.morphs(text, stem=True)
    result = ' '.join(test)
    return result

# 불용어 처리 한 okt
def okt_remove(text):
    tokens = okt.pos(text, stem=True)
    total_words = []
    for word, tag in tokens:
        if tag not in ['Josa'] :
            total_words.append(word)
    result = ' '.join(total_words)
    return result

# 불용어 처리 없이 Mecab 형태소 분석만 진행
def mecab_no_stopwords(text):
    tokens = mecab.morphs(text)
    result = ' '.join(tokens)
    return result

# 불용어 처리 한 mecab
def mecab_remove(text):
    select_token = []
    tokens = mecab.pos(text)
    for word, tag in tokens:
        if tag not in ['JKS', 'EC', 'JKB', 'JX', 'EP', 'NNB', 'VCP', 'ETM']:
            select_token.append(word)
    result = ' '.join(select_token)
    return result

In [6]:
# 데이터 프레임 내용 전체 보기
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

## Okt & 불용어 처리 X

In [17]:
q_s = new_df.question.map(okt_no_stopwords)
okt_df = pd.DataFrame(q_s)
okt_df

,question
0,좋다 아침
1,좋다 아침
2,좋다 아침
3,좋다 아침
4,좋다 아침
5,좋다 아침
6,좋다 아침
7,굿모닝
8,굿모닝
9,굿모닝


## Okt & 불용어 처리 O

In [16]:
q_s = new_df.question.map(okt_remove)
okt_remove_df = pd.DataFrame(q_s)
okt_remove_df

,question
0,좋다 아침
1,좋다 아침
2,좋다 아침
3,좋다 아침
4,좋다 아침
5,좋다 아침
6,좋다 아침
7,굿모닝
8,굿모닝
9,굿모닝


## Mecab & 불용어 처리 X

In [15]:
q_s = new_df.question.map(mecab_no_stopwords)
mecab_df = pd.DataFrame(q_s)
mecab_df

,question
0,좋 은 아침
1,좋 은 아침
2,좋 은 아침
3,좋 은 아침
4,좋 은 아침
5,좋 은 아침
6,좋 은 아침
7,굿모닝
8,굿모닝
9,굿모닝


## Mecab & 불용어 처리 O

In [14]:
q_s = new_df.question.map(mecab_remove)
mecab_remove_df = pd.DataFrame(q_s)
mecab_remove_df

,question
0,좋 아침
1,좋 아침
2,좋 아침
3,좋 아침
4,좋 아침
5,좋 아침
6,좋 아침
7,굿모닝
8,굿모닝
9,굿모닝


# 코사인 유사도 함수

In [18]:
# 코사인 유사도로 정답 구하는 함수
def cos_answer(new_q, new_df):
    tfidf = TfidfVectorizer()
    new_q = pd.Series(new_q)
    all_q = new_df.question.append(new_q)
    tfidf_matrix = tfidf.fit_transform(all_q)
    cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
    questions = cosine_sim[-1][:-1]
    if 1. in questions:
        indices = np.where(questions == 1.)
        indices = indices[0].tolist()
        q_idx = np.random.choice(indices, 1)
        return df.question[q_idx[0]], df.answer[q_idx[0]], questions[q_idx[0]]
    else:
        q_idx = questions.argsort()[-1]
        return df.question[q_idx], df.answer[q_idx], questions[q_idx]

# 형태소, 유사도 별 답변 구하는 함수

In [19]:
def predict(question, tokenize, answer, df):
    real_question = []
    real_answer = []
    sel_q = []
    cos = []

    for q in question:

        real_q = q

        new_q = text_preprocessing(real_q)
        new_q = tokenize(new_q)

        selected_question, selected_answer, cosin = answer(new_q,df)

        real_question.append(real_q)
        real_answer.append(selected_answer)
        sel_q.append(selected_question)
        cos.append(cosin)

    result = pd.DataFrame({
        'question':real_question,
        'selected_question': sel_q,
        'cosine_similarity' : cos,
        'answer':real_answer
    })
    return result

# 전체 공통 질문

In [20]:
new_question = ['굿모닝', '그래 오늘은 뭐할까?', '기운 엄청나', '그래 고마워', '이름이 뭐니?', '나랑 놀자', '대화하면서 노는거 어때', '요새 기분이 들쭉날쭉 해', '그렇다면 다행이네', '조금 못 알아듣는구나',
         '오늘 뭐 먹을까?', '점심 뭐 먹을까', '오늘 날씨 정말 좋네', '오늘 바쁜가 할일이 많아', '오늘따라 시간이 안 가네', '피곤한 하루였어', '너는 기분이 어때?',
         '오늘도 못 자면 어떡하지?', '나 혼자만 있는 것 같아', '오늘따라 우울해', '사는게 재미 없네', '기분이 좋아', '신나는 노래 듣고 싶어', '기분이 별로야', '힘들어',
         '너는 무슨 생각해?', '아니야 괜찮아', '몸이 아파', '허리가 아파', '무릎이 아프다 야', '무슨 생각해?', '그거 좋은 생각이네', '오늘 정말 황당했어', '너무 답답해',
         '오늘따라 스트레스를 많이 받네', '일하기 싫다', '밥을 제대로 못 먹었어', '약을 먹었는데 별 효과 없더라고', '약을 계속 먹어야 한다고 하네', '집중이 안돼', '심심해',
         '요즘 잠이 안와', '슬슬 배가 고파', '오늘 잘 먹었다 배부르다', '요즘 즐겁지 않아', '요새 감정 컨트롤이 잘 안 돼', '누가 나 좀 위로해주면 좋겠다', '눈물 날 거 같아', '잘 잤어?',
         '할머니하고 잘자니까 좋아?']

# 선택한 질문, 답변, 유사도 비교

In [21]:
# okt & 불용어 처리 X
okt_no = predict(new_question, okt_no_stopwords, cos_answer, okt_df)
okt_no

,question,selected_question,cosine_similarity,answer
0,굿모닝,굿 모닝.,1.000000,안녕하세요.
1,그래 오늘은 뭐할까?,그럼 너는 할 수 있어,0.868654,정말 고마워요 힘이 돼요
2,기운 엄청나,기운이 없어,0.578749,힘내세요 제가 있잖아요
3,그래 고마워,"그래, 고마워.",1.000000,천만에요.
4,이름이 뭐니?,너는 이름이 뭐니?,1.000000,저는 이승훈이에요.
5,나랑 놀자,나랑 놀자,1.000000,뭐하고 놀까요?
6,대화하면서 노는거 어때,아무 대화,0.484419,좋아요 말씀해주세요
7,요새 기분이 들쭉날쭉 해,넌 기분이 어때?,0.417284,제게 말을 걸어줘서 기분이 좋아졌어요
8,그렇다면 다행이네,그래야겠어.,0.393813,좋은 생각입니다.
9,조금 못 알아듣는구나,잘 못알아 듣는구나,0.769132,네 아직 부족해요.


In [22]:
# okt & 불용어 처리 O
okt_yes = predict(new_question, okt_remove, cos_answer, okt_remove_df)
okt_yes

,question,selected_question,cosine_similarity,answer
0,굿모닝,굿 모닝.,1.000000,반갑습니다.
1,그래 오늘은 뭐할까?,그럼 너는 할 수 있어,0.868654,정말 고마워요 힘이 돼요
2,기운 엄청나,기운이 없어,0.578749,힘내세요 제가 있잖아요
3,그래 고마워,"그래, 고마워.",1.000000,천만에요.
4,이름이 뭐니?,이름이 뭐야?,1.000000,저는 정산하에요.
5,나랑 놀자,나랑 놀자,1.000000,뭐하고 놀까요?
6,대화하면서 노는거 어때,아무 대화,0.484419,좋아요 말씀해주세요
7,요새 기분이 들쭉날쭉 해,넌 기분이 어때?,0.417284,제게 말을 걸어줘서 기분이 좋아졌어요
8,그렇다면 다행이네,"응, 그럴게.",0.486579,좋은 생각이에요.
9,조금 못 알아듣는구나,잘 못알아 듣는구나,0.769132,네 아직 부족해요.


In [23]:
# Mecab & 불용어 처리 X
mecab_no = predict(new_question, mecab_no_stopwords, cos_answer, mecab_df)
mecab_no

,question,selected_question,cosine_similarity,answer
0,굿모닝,굿 모닝.,1.000000,안녕하십니까?
1,그래 오늘은 뭐할까?,"그래, 오늘은 쉬었다가 해야겠다.",0.471111,"네, 쉬었다가 하세요."
2,기운 엄청나,기운이 없어,0.663465,힘내세요 제가 있잖아요
3,그래 고마워,"그래, 고마워.",1.000000,별 말씀을요.
4,이름이 뭐니?,너는 이름이 뭐야?,1.000000,마스터입니다.
5,나랑 놀자,그런데 증상이 나빠진 거 같아.,0.000000,너무 심하시면 병원을 다시 가보는 건 어떨까요?
6,대화하면서 노는거 어때,그냥 대화하면서 놀자,0.747202,대화하면서 노는거 재밌어요
7,요새 기분이 들쭉날쭉 해,그런데 요새는 사이가 안 좋거든.,0.386995,한번 틀어진 사이를 다시 되돌리는 것만큼 어려운 일은 없는 것 같아요.
8,그렇다면 다행이네,그렇구나 푹 쉬어,0.321676,네 감사해요
9,조금 못 알아듣는구나,잘 못알아 듣는구나,0.745551,네 아직 부족해요.


In [24]:
# Mecab & 불용어 처리 O
mecab_yes = predict(new_question, mecab_remove, cos_answer, mecab_remove_df)
mecab_yes

,question,selected_question,cosine_similarity,answer
0,굿모닝,굿 모닝.,1.000000,안녕하세요!
1,그래 오늘은 뭐할까?,"그래, 오늘은 쉬었다가 해야겠다.",0.547388,기운 내세요.
2,기운 엄청나,기운이 없어,0.663465,힘내세요 제가 있잖아요
3,그래 고마워,"그래, 고마워.",1.000000,천만에요.
4,이름이 뭐니?,너는 이름이 뭐야?,1.000000,박현수에요.
5,나랑 놀자,그런데 증상이 나빠진 거 같아.,0.000000,너무 심하시면 병원을 다시 가보는 건 어떨까요?
6,대화하면서 노는거 어때,그냥 대화하면서 놀자,0.638474,대화하면서 노는거 재밌어요
7,요새 기분이 들쭉날쭉 해,다 맘에 안 들고 기분이 안 좋아요.,0.426169,"정도가 심하다면, 의사와 상담을 하는 것도 좋은 방법이에요."
8,그렇다면 다행이네,그런데 살아보니까 그렇지도 않아요.,0.473412,그런 일이 있으셨군요.
9,조금 못 알아듣는구나,잘 못알아 듣는구나,0.603358,네 아직 부족해요.
